In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install requirements.txt

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","+z8wCDJ=3Z-nDAoZExt*"),
    ca_certs="http_ca.crt 인증서 위치"
)
es.ping()

In [ ]:
import pandas as pd

df = pd.read_csv("alltagging20_tfidf20_test.csv")

In [ ]:
df.head()

In [ ]:
for index, keyword in enumerate(keywords):
    print(keyword)

In [ ]:
df = df[['Column1', '상품이름', '가격', '상품정보','태깅num']]

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
df["DescriptionVector"] = df["상품정보"].apply(lambda x: model.encode(x))

In [ ]:
indexMapping = {
    "properties":{  
        "ProductID":{
            "type" : "text"
        },
        "ProductName":{
            "type":"text"
        },
        "Price (INR)":{
            "type":"long"
        },
        "Description":{
            "type":"text"
        },
        "taggingNum":{
            "type":"long"
        },
        "DescriptionVector":{
            "type":"dense_vector",
            "dims": 768,
            "index":True,
            "similarity": "cosine"
        }

    }
}

In [ ]:
#from indexMapping import indexMapping

es.indices.create(index="tagging_with", mappings=indexMapping)

In [ ]:
record_list = df.to_dict("records")

In [ ]:
record_list

In [ ]:
for record in record_list:
    try:
        es.index(index="tagging_with", document=record)
    except Exception as e:
        print(e)

In [ ]:
es.count(index="tagging_with")